In [1]:
# 引入所需要的工具库 
from imutils.object_detection import non_max_suppression
import numpy as np
import imutils
from imutils.video import FPS
from imutils.video import VideoStream 
import pytesseract
import argparse
import cv2
import time
from utils import decode_predictions

In [2]:
# 定义要提取 EAST 网络的输出层名字。包括有两个输出：
# 第一个输出是文字检索框的概率
# 第二个输出是文字检索框的几何信息
layerNames = ["feature_fusion/Conv_7/Sigmoid",
              "feature_fusion/concat_3"]

# 读取 EAST 网络
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(r"frozen_east_text_detection.pb")

# 定义输入网络视频图像的大小（必须为 32 的倍数）
newW, newH = 160, 160

# 开启摄像头
vs = cv2.VideoCapture(0)

while True:
    
    _, frame = vs.read()
    
    if frame is None:
        break
        
    orig = frame.copy()
    
    (H, W) = frame.shape[:2]
    rW = W / float(newW)
    rH = H / float(newH)
    
    # 预处理输入网络的视频图像
    blob = cv2.dnn.blobFromImage(frame, 1.0, (newW, newH),
        (123.68, 116.78, 103.94), swapRB=True, crop=False)
    
    # 将处理后的图像送入到 EAST 网络中。
    net.setInput(blob)
    
    # 得到网络的输出
    scores, geometry = net.forward(layerNames)
    
    # 通过 decode_predictions 函数得到文字检测的水平候选框
    rects, confidences = decode_predictions(scores, geometry)
    
    # 通过 non_max_suppression 函数来抑制非最大值后选框，去除冗余的文字框。
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    
    # 分析每一个文字框中的文字识别
    for (startX, startY, endX, endY) in boxes:
        
        # 将文字框几何信息坐标映射到输入视频图像原尺寸。
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        
        # 检查文字框的边缘信息并合理处理
        startX = max(0, startX)
        startY = max(0, startY)
        endX = min(W, endX)
        endY = min(H, endY)
        
        # 提取文字框区域，准备进行文字识别
        roi = orig[startY:endY, startX:endX]
        
        # 利用 pytesseract 进行 OCR
        config = ("-l eng --oem 1 --psm 7")
        text = pytesseract.image_to_string(roi, config=config)
        
        # 显示所识别的文字
        print("OCR TEXT")
        print("========")
        print("{}\n".format(text))
        
        # 剔除无效字符
        text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
        
        # 将识别的文字实时显示在视频图像中。
        cv2.rectangle(orig, (startX, startY), (endX, endY),
            (228, 179, 2), 2)
        cv2.putText(orig, text, (startX, startY - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
    # 显示视频图像
    cv2.imshow("Text Detection", orig)
    
    # 设置视频流间隔
    key = cv2.waitKey(10) & 0xFF

    # 设置退出键为 `q`
    if key == ord("q"):
        break

vs.release()
cv2.destroyAllWindows()


[INFO] loading EAST text detector...
OCR TEXT
+e PARE

OCR TEXT
+e EAR ES

OCR TEXT
2a eae

OCR TEXT
teEAR ES

OCR TEXT
te eARE)

OCR TEXT
eae el

OCR TEXT
he

OCR TEXT
ees 8

OCR TEXT
:

OCR TEXT
ri

OCR TEXT
<_

OCR TEXT
a

OCR TEXT
:

OCR TEXT
he el

OCR TEXT
tee Lee!

OCR TEXT
tee Lee!

OCR TEXT
tee ARE)

OCR TEXT
eh el

OCR TEXT
+e EAR ES

OCR TEXT
el

